# Trabalho de Introdução à Ciência de Dados - Equity in Athletics (Equidade no Atletismo)

## Membros
Cecília Junqueira - 2022094888

Carolina Pilla -2022043299


Felipe Gomide - 2022043213

Lucas Junqueira - 2022043264

## Informações 

* Base de dados do Departamentodde Educação dos E.U.A.: o [Equity in Athletics](https://ope.ed.gov/athletics/#/datafile/list) -  2006 à 2021
* A base engloba dados sobre equipes esportivas em faculdades americanas, considerando 43 esportes. Nosso foco é na receita (lucro e gastos) das faculdades e nas proporções de homens e mulheres.st

## Referências
* Best D1 Colleges. Disponível em: &#60;https://www.ncsasports.org/best-colleges/best-division-1-colleges&#62;. Acesso em: 16 ago. 2024. 
* 

## Setup Inicial

Instalação de biblioteca auxiliar e imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
#from adjustText import adjust_text
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score

# Processamento dos Dados

## União dos dados de todos os anos

In [ ]:
df = pd.read_excel("https://github.com/FelipeGomide/ICD-EquityInAthletics/raw/master/dados/EADA_2006.xls")
df['YEAR'] = 2006

to_concat = []
to_concat.append(df)
for ano in range(2007, 2012):
    df_aux = pd.read_excel(f"https://github.com/FelipeGomide/ICD-EquityInAthletics/raw/master/dados/EADA_{ano}.xls")
    df_aux['YEAR'] = ano

    to_concat.append(df_aux)

for ano in range(2012, 2022):
    df_aux = pd.read_excel(f"https://github.com/FelipeGomide/ICD-EquityInAthletics/raw/master/dados/EADA_{ano}.xlsx")
    df_aux['YEAR'] = ano

    to_concat.append(df_aux)

df = pd.concat(to_concat)
df

## Seleção dos atributos de interesse

In [ ]:
#Seleção das colunas de interesse
aux = df[['SUM_PARTIC_MEN', 'SUM_PARTIC_WOMEN', 'Sports', 'YEAR', 'state_cd', 'TOTAL_OPEXP_INCLCOED', "SUM_TOTAL_HDCOACH", "SUM_TOTAL_ASSTCOACH", "TOTAL_REVENUE_ALL", "TOTAL_EXPENSE_ALL"]].reset_index(drop=True)

# Nova coluna, total de participantes
aux['TOTAL_PARTIC'] = aux['SUM_PARTIC_MEN'] + aux['SUM_PARTIC_WOMEN']

aux.shape

In [ ]:
# Modalidades com erro de digitação ou alguma mudança de categorização a partir dos anos
rename = {"Track and Field and Cross Country (combined)": "All Track Combined",
          "Swimming and Diving (combined)": "Swimming and Diving",
          "Field hocky": "Field Hockey"}


# Modalidades que já estão contidas em algum grupo serão removidas
to_drop = ["Track and Field, Indoor", "Track and Field, Outdoor", "Track and Field, X-Country",
           "Track and Field (Indoor)", "Track and Field (Outdoor)", "Swimming",
           "Synchronized Swimming", "Synchronized swimming", "Diving", "Cross Country",
            "Team Handball", "Table Tennis", "Badminton", "Archery", "Weight Lifting", "Rifle", # Esta linha: modalidades com menos de 10 mil atletas todos os anos
           "Other Sports"] # Conjunto de esportes, sem muita informação, possivelmente muito variados

aux = aux.replace(rename)
aux = aux[~aux['Sports'].isin(to_drop)]

In [ ]:
data = aux.groupby(by=['YEAR', 'Sports', 'state_cd']).sum().reset_index()
data.to_csv("tabelas_pre_processadas/modalidades_por_estado.csv", index=False)

Essa tabela será adicionada ao GitHub, para evitar o tempo de processamento de todos os dados, durante a realização das análises.

# Pergunta 1: Quais são as modalidades esportivas com maiores investimentos em comissão técnica e contratações?

Após uma análise mais minuciosa do conteúdo das tabelas, percebi que não existe informação suficiente pra quantificar esse tipo de investimento, apenas investimento no geral. Por isso, tive o enfoque nas diferenças de investimento total e no número de participantes de cada modalidade esportiva.

## Importando as tabelas

Tabela processada anteriormente do GitHub e tabela auxiliar com informações geográficas para desenho dos mapas.

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/FelipeGomide/ICD-EquityInAthletics/master/tabelas_pre_processadas/modalidades_por_estado.csv")
data = data.drop(columns = 'Unnamed: 0')

states = geopandas.read_file('https://raw.githubusercontent.com/joncutrer/geopandas-tutorial/master/data/usa-states-census-2014.shp')
states = states[['STUSPS','geometry','NAME']]

states['coords'] = states['geometry'].apply(lambda x: x.representative_point().coords[:])
states['coords'] = [coords[0] for coords in states['coords']]

states = states.drop_duplicates()
states

## Análise Exploratória: Plotando mapas e gráficos de evolução no tempo

### Geração de Mapas, quantidade de participantes por modalidade

In [ ]:
def plot_modalidade(states, aux, modalidade, ano):

    # Seleciono o ano e esporte de interesse no dataframe
    esporte = aux.loc[aux['Sports'] == f'{modalidade}']
    modal = esporte.loc[esporte['YEAR'] == ano]

    #Agrego os dados de mapas no dataframe
    map = states.merge(modal, left_on='STUSPS', right_on='state_cd', how='left')
    map = map[['geometry', 'TOTAL_PARTIC', 'STUSPS', 'NAME', 'coords']]

    f, ax = plt.subplots()
    f.set_size_inches(14, 12)

    map.plot(
        ax=ax,
        column='TOTAL_PARTIC',
        cmap='Oranges',
        edgecolor="black",
        linewidth=0.4,
        missing_kwds={
            "color": "lightgrey",
            "edgecolor": "black",
            "label": "Missing values",
        },
    )

    texts = []

    for idx, row in map.iterrows():
        texts.append(ax.text(row.coords[0], row.coords[1], s=row['NAME'], horizontalalignment='center',)) #bbox={'facecolor': 'white', 'alpha':0.8, 'pad': 2, 'edgecolor':'none'})

    sm = plt.cm.ScalarMappable(
        cmap="Oranges",                      # Usa o mesmo cmap do mapa
        norm=plt.Normalize(
            vmin= 0,  # Valor mínimo
            vmax=map["TOTAL_PARTIC"].max(),  # Valor máximo
        ),
    )

    cax = f.add_axes(
        [
            0.9,    # posicao x (entre 0.0 e 1.0)
            0.3,    # posicao y (entre 0.0 e 1.0)
            0.03,    # largura x
            0.40,    # altura y
        ]
    )

    ax.set_title(f'Quantity of Athletes by State, {modalidade} {ano}', fontdict={"fontsize": 16})
    ax.axis('off')

    # A linha abaixo utiliza a biblioteca que otimiza o posicionamento dos textos no plot
    # Melhora bastante a visualização dos nomes, já que evita sobreposição,
    # porém afeta bastante o desempenho
    #adjust_text(texts, time_lim=30, arrowprops=dict(arrowstyle='-', color='black', alpha=1), ax=ax)

    f.colorbar(sm, cax=cax)
    plt.show()

# TO-DO imprimir os gráficos bonitinho contando os resultados

In [ ]:
modalidades = data['Sports'].drop_duplicates().sort_values().reset_index(drop=True)
for sport in modalidades:
    plot_modalidade(states, data, sport, 2017)

### Gráficos de Evolução das Modalidades ao passar do tempo

In [ ]:
linhas = data.groupby(by=['YEAR', 'Sports']).sum().reset_index()
linhas = linhas.loc[:, ['YEAR', 'Sports', 'TOTAL_PARTIC']]
linhas = linhas.sort_values(by=['Sports', 'YEAR'])

esportes = linhas.groupby('Sports')

fig, ax = plt.subplots(figsize=(14, 8))

texts = []

for sport, info in esportes:
    ax.plot(info['YEAR'], info['TOTAL_PARTIC'], marker='.', linestyle='-', label=sport)

    ultimo_participantes = info.iloc[-1]['TOTAL_PARTIC']

    texts.append(ax.text(info.iloc[-1]['YEAR'], ultimo_participantes, sport, horizontalalignment='center',))

    #ax.annotate(sport, xy=(info.iloc[-1]['YEAR'], ultimo_participantes), xytext=(10, 0),
                #textcoords='offset points', ha='left', va='baseline')

ax.set_xlabel('Ano')
ax.set_ylabel('Participantes')
ax.set_title('Evolução do Número de Participantes por Esporte')
#ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

#adjust_text(texts, time_lim=30, arrowprops=dict(arrowstyle='-', color='black', alpha=1), ax=ax)

# Exibindo o gráfico
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
linhas = data.groupby(by=['YEAR', 'Sports']).sum().reset_index()
linhas = linhas.loc[:, ['YEAR', 'Sports', 'TOTAL_EXPENSE_ALL']]
linhas = linhas.sort_values(by=['Sports', 'YEAR'])

esportes = linhas.groupby('Sports')

fig, ax = plt.subplots(figsize=(14, 8))

for sport, info in esportes:
    ax.plot(info['YEAR'], info['TOTAL_EXPENSE_ALL'], marker='.', linestyle='-', label=sport)

    ultimo_participantes = info.iloc[-1]['TOTAL_EXPENSE_ALL']
    ax.annotate(sport, xy=(info.iloc[-1]['YEAR'], ultimo_participantes), xytext=(10, 0),
                textcoords='offset points', ha='left', va='center')

ax.set_xlabel('Ano')
ax.set_ylabel('Investimento total em Bilhões')
ax.set_title('Evolução do Investimento Total por Esporte')
#ax.legend(loc='center left', bbox_to_anchor=(1.1, 0.5))

# Exibindo o gráfico
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
linhas = data.groupby(by=['YEAR', 'Sports']).sum().reset_index()
linhas = linhas.loc[:, ['YEAR', 'Sports', 'TOTAL_EXPENSE_ALL']]
linhas = linhas.sort_values(by=['Sports', 'YEAR'])

esportes = linhas.groupby('Sports')

fig, ax = plt.subplots(figsize=(14, 8))

for sport, info in esportes:
    ax.plot(info['YEAR'], info['TOTAL_EXPENSE_ALL'], marker='.', linestyle='-', label=sport)

    ultimo_participantes = info.iloc[-1]['TOTAL_EXPENSE_ALL']
    ax.annotate(sport, xy=(info.iloc[-1]['YEAR'], ultimo_participantes), xytext=(10, 0),
                textcoords='offset points', ha='left', va='center')

ax.set_xlabel('Ano')
ax.set_ylabel('Investimento total em Centenas de Milhões')
ax.set_title('Evolução do Investimento Total por Esporte, recorte até meio bilhão em investimento')
ax.set_ylim(0, 500_000_000)
#ax.legend(loc='center left', bbox_to_anchor=(1.1, 0.5))

# Exibindo o gráfico
plt.grid(True)
plt.tight_layout()
plt.show()

### Resultados

Observamos que na questão de número de atletas, existem várias modalidades que dividem o total de atletas, porém o atletismo é a líder nesse quesito, já que é um compilado de muitas modalidades.

Apesar disso, o gráfico de investimento mostra um cenário bem diferente e muito mais desigual.
Futebol Americano e Basquete dominam o cenário em todos os anos. O Atletismo que antes era o primeiro em número de atletas, tem cerca de 12% do investimento do Futebol Americano.

É perceptível que o número de atletas tem uma certa regularidade através dos anos, os números não aumentam muito, porém o investimento tem crescido a cada ano, o que significa que o investimento por atleta tem aumentado bastante.

Todos os valores sofreram uma queda em 2020, provavelmente em decorrência da pandemia de Covid-19.

## Intervalo de Confiança

In [ ]:
def bootstrap_mean(df, column, n=5000, size=None):
    '''
    Faz um boostrap da média da coluna escolhida. Gera amostras.

    Parâmetros
    ----------
    df: o dataframe
    column: a coluna que queremos focar
    n: número de amostras para o bootstrap
    size: tamanho de cada amostra, por padrão vira o tamanho do df.
    '''
    if size is None:
        size = len(df)
    values = np.zeros(n)
    for i in range(n):
        sample = df[column].sample(size, replace=True)
        values[i] = sample.mean()
    return values

### A liderança em popularidade do Futebol Americano pode ser explicada pelo acaso?

O boxplot que mostra os intervalos de confiança mostra que não, o intervalo de bootstrap do Futebol Americano é muito distinto dos outros esportes, ou seja, do ponto de vista estatístico, existe um claro viés que influencia no investimento do esporte. 

O plot seguinte mostra a mesma relação, porém analisada sobre os investimentos em atletismo, este mostra que o atletismo está contido no intervalo de confiança, logo o acaso explica seus valores de investimento.

In [ ]:
football = data.loc[data["Sports"] == "Football"]
no_football = data.loc[data["Sports"] != "Football"]

football_mean = bootstrap_mean(football, "TOTAL_EXPENSE_ALL")
all_mean = bootstrap_mean(no_football, "TOTAL_EXPENSE_ALL")

to_plot = pd.DataFrame()
to_plot['Football'] = football_mean
to_plot['All without Football'] = all_mean

to_plot.boxplot(grid=False, sym='', whis=[5, 95], showmeans=True)

In [ ]:
atletismo = data.loc[data["Sports"] == "All Track Combined"]
all = data.loc[data["Sports"] != "All Track Combined"]

atletismo_mean = bootstrap_mean(atletismo, "TOTAL_EXPENSE_ALL")
all_mean = bootstrap_mean(all, "TOTAL_EXPENSE_ALL")

to_plot = pd.DataFrame()
to_plot['All Track Combined'] = atletismo_mean
to_plot['All Sports without Track and Field'] = all_mean

to_plot.boxplot(grid=False, sym='', whis=[5, 95], showmeans=True)